In [2]:
import warnings
import os
from src.green_pont import load_json_with_comments
from src.urban_wind import wind_speed_grib, read_cfd_wind, scale_cfd_wind, save_local_wind, plot_maps
from src.plots_jupyter import plot_urban_wind
import ipywidgets as widgets

warnings.filterwarnings("always")

In [3]:
cf=load_json_with_comments('etc/settings_wind_maps.json')

path_cfd=cf['path_cfd'] #'/projects/urbanair/DATA/DATA_AntwerpWindStudy/wind_ratios'
angles=cf["angles"]
height=cf["height"]
path = cf['path_meteo'] #'data/meteo_input/'
grib_file= cf['grib_file'] # 'GRIBPFDEOD+$hour$h00m00s'
output_path=cf["output_path"]

masks=[cf['geometry']["buildings"], cf['geometry']["waterbodies"]]  

# read meteo file from meso-scale model
wind_meteo = wind_speed_grib(30, path, grib_file)

Reading GRIB files from  data/meteo_input/


Can't create file 'data/meteo_input/GRIBPFDEOD+0000h00m00s.9093e.idx'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/cfgrib/messages.py", line 535, in from_indexpath_or_filestream
    self = cls.from_fieldset(filestream, index_keys, computed_keys)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/cfgrib/messages.py", line 378, in from_fieldset
    return cls.from_fieldset_and_iteritems(fieldset, iteritems, index_keys, computed_keys)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/cfgrib/messages.py", line 391, in from_fieldset_and_iteritems
    for field_id, raw_field in iteritems:
  File "/opt/conda/lib/python3.11/site-packages/cfgrib/messages.py", line 291, in __iter__
    for message in self.itervalues():
  File "/opt/conda/lib/python3.11/site-packages/cfgrib/messages.py", line 267, in iterval

FileNotFoundError: [Errno 2] No such file or directory: 'data/meteo_input/GRIBPFDEOD+0000h00m00s'

In [3]:
# read pre-computed normalized CFD wind ratios
cfd_ratio=read_cfd_wind(path_cfd,angles, height)

TypeError: read_cfd_wind() missing 1 required positional argument: 'crop_bounds'

In [ ]:
# scale meso-scale wind to local urban scale using CFD ratios
wind_local=scale_cfd_wind(wind_meteo, cfd_ratio)

os.makedirs(output_path, exist_ok=True)

saved_files=save_local_wind(wind_local, cfd_ratio, output_path, reproject=True, mask_frames=masks)

#plot_maps(saved_files, path=output_path)    

In [ ]:
# Define sliders
slider1 = widgets.IntSlider(
    value=1, min=0, max=24, step=1,
    description='Hour of the day:', continuous_update=True
)

# Display all three
display(slider1)

In [ ]:

print(str.zfill(str(slider1.value), 2))

hour=str.zfill(str(slider1.value), 2)
tif_path="data/maps/urban_wind//wind_175_20240811_"+hour+"00_4326.tif"
print(tif_path)

#tif_path = "/projects/urbanair/urbanair_greenpotential/data/maps/recomendation_method3_4326.tif"
plot_urban_wind(tif_path, vmin=0, vmax=0.1, mask_shp="data/geometry/bld_all_fused.shp")